In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ampl_utils import ampl_license
from amplpy import AMPL, ampl_notebook

from models import MixedStrategyMasterProblem, to_set_ids, MathematicalModel, values_to_list, AttackGenerationPricingProblem

from dataloader import Network

from itertools import product

solver='cplex'
ampl_license_uuid = ampl_license()

ampl = ampl_notebook(
    modules=[solver],
    license_uuid=ampl_license_uuid)

Licensed to AMPL Academic Community Edition License for <s207399@student.pg.edu.pl>.


In [3]:
%%writefile models/cpop.mod

# Set of attacks
set ATTACKS;
set VERTICES;
# Set of nodes to attack given attack a
set V_A {ATTACKS} within VERTICES;
# C(a) is the set of components resulting from an attack a
## Component Ids hold the indices of components resulting from attack a
set COMPONENT_IDS {ATTACKS};
set C_A {a in ATTACKS, COMPONENT_IDS[a]} within VERTICES;

# How many controllers to use
param M;

var Y >= 0;
var s {VERTICES} binary;
var y {VERTICES, ATTACKS} binary;

maximize SurvivingNodes:
    Y;

s.t. NumberOfControllers:
    sum {v in VERTICES} s[v] = M;

s.t. ZeroAttackedNodes {a in ATTACKS, v in V_A[a]}:
    y[v, a] = 0;

s.t. ZeroComponentsWithoutControllers {a in ATTACKS, c_id in COMPONENT_IDS[a]}:
    sum {v in C_A[a, c_id]} y[v, a] <= card(C_A[a, c_id]) * sum {v in C_A[a, c_id]} s[v];

s.t. NumberOfSurvivingNodes {a in ATTACKS}:
    Y <= sum {v in VERTICES} y[v, a];

Overwriting models/cpop.mod


In [12]:
class CPOP(MathematicalModel):
    def __init__(self, network):
        self._model_file = 'models/cpop.mod'
        super(CPOP, self).__init__(self._model_file)
        self._name = 'CPOP'

        self.network = network

    def report(self):
        if not self._solved:
            self.solve()

        var_s = self.ampl.get_variables()['s']
        var_y = self.ampl.get_variables()['y']
        # y = values_to_list(var_y)
        # print(var_y.get_values())
        s_star = values_to_list(var_s)
        return {
            's': s_star,
        }

    def load_data(self, attacks, M):
        super(CPOP, self).load_data()
        vertex_list = list(self.network.nodes)

        # Key => attack set pair
        attack_dict = to_set_ids(attacks)

        self.ampl.set['VERTICES'] = vertex_list
        self.ampl.set['ATTACKS'] = attack_dict.keys()

        V_A = {}
        for i, a in attack_dict.items():
            # vs = self.network.remaining_nodes(a)
            V_A[i] = a
        self.ampl.set['V_A'] = V_A

        component_ids = {}
        C_A = {}
        for a, attack in attack_dict.items():
            components = self.network.attack(attack)
            component_ids[a] = list(range(len(components)))
            for i, c in enumerate(components):
                C_A[(a, i)] = list(c)
        print(C_A)

        self.ampl.set['COMPONENT_IDS'] = component_ids
        self.ampl.set['C_A'] = C_A

        self.ampl.param['M'] = M

In [13]:
cpop = CPOP(Network('dognet'))
cpop.load_data([[0, 1], [5, 6]], 3)
cpop.report()

{(0, 0): [2, 3, 4, 5, 6, 7, 8, 9], (1, 0): [0, 1, 2, 3, 4, 7, 8], (1, 1): [9]}
CPLEX 22.1.2: optimal solution; objective 8
0 simplex iterations


{'s': [0, 0, 1, 0, 0, 0, 1, 0, 0, 1]}